In [0]:
from pyspark.sql import functions as F

# Load sample Delta table
df = spark.read.format("delta").load(
    "dbfs:/databricks-datasets/nyctaxi-with-zipcodes/subsampled"
)

clean = (
    df.select(
        "tpep_pickup_datetime",
        "tpep_dropoff_datetime",
        "trip_distance",
        "fare_amount",
        "pickup_zip",
        "dropoff_zip",
    )
    .filter(
        (F.col("fare_amount") > 0) &
        (F.col("trip_distance") > 0)
    )
)

# Write to a managed table (no path needed)

# Create the database if it does not exist
spark.sql("CREATE DATABASE IF NOT EXISTS dbx_demo")

# Overwrite the managed table directly
clean.write.format("delta").mode("overwrite").saveAsTable("dbx_demo.nyc_taxi")

display(spark.table("dbx_demo.nyc_taxi").limit(10))
